### A. Import dữ liệu cần thiết

In [21]:
import pandas as pd
import numpy as np
import plotly.express as px
import dash
from collections import Counter
from dash import  html, dcc, Input, Output, dash_table, State

In [22]:
trending_videos = pd.read_csv('Data/trending_videos.csv', index_col=False)
trending_videos_unique = pd.read_csv('Data/trending_videos_unique.csv', index_col=False)

### B. Trực quan hoá

#### 1. Phân tích tổng quan

##### a. Thời điểm đăng trong ngày của video

In [23]:
upload_time = trending_videos_unique[['published_at']].copy()
upload_time['published_at'] = pd.to_datetime(upload_time['published_at'])
upload_time['hour'] = upload_time['published_at'].dt.hour
time_ranges = [f'{i}h - {i+1}h' for i in range(24)]
video_counts = [upload_time[(upload_time['hour'] >= i) & (upload_time['hour'] < i+1)].shape[0] for i in range(24)]
time_video_df = pd.DataFrame({'Time Range': time_ranges, 'Video Count': video_counts})

##### b. Thời gian tồn tại trên trending

In [24]:
time_on_trending = trending_videos_unique[['trending_time']].copy()
time_on_trending = time_on_trending.groupby('trending_time').size().reset_index(name='frequency')
time_on_trending['trending_time'] = pd.to_timedelta(time_on_trending['trending_time'])
time_on_trending = time_on_trending.sort_values(by='trending_time')

In [25]:
time_on_trending['days_on_trending'] = time_on_trending['trending_time'].dt.days

#### 2. Phân tích theo chủ đề

##### a. Phần trăm của từng chủ đề trên trending

In [26]:
category_counts = trending_videos['category'].copy().value_counts()
category_full = category_counts.reset_index()

In [27]:
threshold = 400
category_pie = category_full.copy()
category_pie['category'] = category_pie['category'].apply(lambda x: x if category_counts[x] >= threshold else 'Others')
category_pie = category_pie.groupby('category').sum()

##### b. Trung bình lượt tương tác theo chủ đề

In [28]:
pd.options.display.float_format = None
general_category = trending_videos[['category', 'view_count', 'like_count', 'comment_count']]
general_category = general_category.groupby('category').mean().reset_index()
general_category['like_to_view'] = (general_category['like_count'] / general_category['view_count']) * 100
general_category['comment_to_view'] = (general_category['comment_count'] / general_category['view_count'])* 100

##### c. Thời gian trending trung bình của chủ đề

In [29]:
trending_category = trending_videos_unique[['category', 'trending_time']].copy()
trending_category['trending_time'] = pd.to_timedelta(trending_category['trending_time'])
trending_category = trending_category.groupby('category').mean()
trending_category = trending_category / np.timedelta64(1, 'h')

In [30]:
trending_category = trending_category.sort_values('trending_time')

#### 3. Phân tích theo nội dung chi tiết

##### a. Từ khoá trong tag

In [31]:
video_tags = trending_videos_unique['video_tags'].copy()
video_tags = video_tags.dropna()
tags_list = []
for tagline in video_tags:
    tags_list.extend(tagline.split(','))
count_tags = Counter(tags_list)
tags_df = pd.DataFrame(count_tags.items(),columns=['tag','count'])
tags_df_sort = tags_df.sort_values('count',ascending=True).tail(25)

#### 4. Phân tích theo thời gian

##### a. Độ dài video theo thời gian

In [32]:
duration_videos = trending_videos[['snapshot_date', 'duration']].copy()
duration_videos['week'] = pd.to_datetime(duration_videos['snapshot_date']).dt.to_period('W')
bins = [pd.Timedelta(minutes=0), pd.Timedelta(minutes=1), pd.Timedelta(minutes=10), pd.Timedelta(minutes=60), pd.Timedelta(hours=24)]
labels = ['<1p', '1-10p', '10-60p', '>60p']
duration_videos['duration_time'] = pd.cut(duration_videos['duration'], bins=bins, labels=labels, right=False)
duration_videos = duration_videos.groupby(['week', 'duration_time'], observed=False).size().unstack(fill_value=0)
duration_videos = duration_videos.reset_index()

for col in duration_videos.columns[1:]:
    duration_videos[col] = duration_videos[col].cumsum()


In [33]:
duration_videos['week'] = duration_videos['week'].astype(str)
duration_videos_long = duration_videos.melt(id_vars='week', 
											var_name='time', 
											value_name='count')

##### b. Chủ đề theo thời gian

In [34]:
video_counts = 300
category_area = trending_videos[['snapshot_date', 'category']].copy()
category_area['category'] = category_area['category'].apply(lambda x: x if category_counts[x] >= video_counts else 'Others')
category_area = category_area.groupby(['snapshot_date', 'category']).size().unstack(fill_value=0)
category_area.index = pd.to_datetime(category_area.index)
category_area['month'] = category_area.index.to_period('W')
category_area = category_area.groupby('month').sum()
category_area = category_area.reset_index()

In [35]:
for col in category_area.columns[1:]:
    category_area[col] = category_area[col].cumsum()

In [36]:
category_area['month'] = category_area['month'].astype(str)
category_area_long = category_area.melt(id_vars='month', 
                                        var_name='categories', 
                                        value_name='count')
category_area_long = category_area_long.sort_values(by=['month', 'count'], ascending=[True, False])

##### c. Trung bình tương tác theo thời gian

In [37]:
general_videos = trending_videos[["view_count", "like_count", "comment_count"]].copy()

In [38]:
general_month = trending_videos[["snapshot_date", "view_count", "like_count", "comment_count"]].copy()

general_month['snapshot_date'] = pd.to_datetime(general_month['snapshot_date'])

general_month['snapshot_date'] = general_month['snapshot_date'].dt.to_period('M')

general_month = general_month.groupby('snapshot_date').mean().reset_index()


### Dash

In [39]:
app = dash.Dash(__name__)

app.layout = html.Div([
    dcc.Dropdown(
        id='chart-dropdown',
        options=[
            {'label': 'Số video theo giờ', 'value': 'hourly'},
            {'label': 'Biểu đồ phân phối video', 'value': 'other0'},
            {'label': 'Tỉ lệ phần trăm của từng chủ đề', 'value': 'other1'},
            {'label': 'Biểu đồ Tương tác Trung bình Theo Danh mục', 'value': 'other2'},
            {'label': 'Thời gian trending trung bình của chủ đề', 'value': 'other3'},
            {'label': 'Từ khoá trong tag', 'value': 'other4'},
            {'label': 'Độ dài video theo thời gian', 'value': 'other6'},
            {'label': 'Chủ đề theo thời gian', 'value': 'other7'},
            {'label': 'Trung bình tương tác theo thời gian', 'value': 'other8'},
        ],
        value='hourly'
    ),
    html.Div([
        dcc.RadioItems(
            id='interaction-selector',  
            options=[],  
            value='like_to_view',
            inline=True,
            style={'margin-top': '10px', 'display': 'none'}  
        )
    ], id='interaction-selector-container'),
    html.Div([
        html.Div([dcc.Graph(id='graph', style={'height': '550px', 'width': '95%'})], className="six columns"),
        html.Div([dash_table.DataTable(
            id='data_table',
            columns=[],
            data=[],
            style_table={'height': '300px', 'overflowY': 'auto'},
            style_cell={'textAlign': 'center', 'padding': '5px'},
            style_header={'backgroundColor': 'lightgrey', 'fontWeight': 'bold'}
        )], id='data_table-container', className="six columns", style={'display': 'block'})
    ], className="row")
])


@app.callback(
    [Output('graph', 'figure'),
     Output('data_table', 'columns'),
     Output('data_table', 'data'),
     Output('interaction-selector', 'style'),
     Output('interaction-selector', 'options'),
     Output('data_table-container', 'style')],
    [Input('chart-dropdown', 'value'),
     Input('interaction-selector', 'value')],
    State('interaction-selector', 'value')
)


def update_graph(selected_chart, selected_metric, current_metric):
    interaction_options = []
    interaction_style = {'display': 'none'}
    table_style = {'display': 'block'}
    fig = {}
    columns = []
    table_data = []


    if selected_chart == 'hourly':
        fig = px.bar(
            time_video_df, 
            x='Time Range', 
            y='Video Count', 
            title='Số lượng video được tải lên trong từng khoảng thời gian',
            labels={'Time Range': 'Khoảng Thời Gian', 'Video Count': 'Số Lượng Video'},
            color='Video Count',
            color_continuous_scale='Viridis'
        ).update_layout(xaxis={'tickangle': 45}, showlegend=False)

        columns = [{"name": col, "id": col} for col in time_video_df.columns]
        table_data = time_video_df.to_dict('records')


    elif selected_chart == 'other0':
        fig = px.bar(
            time_on_trending, 
            x='days_on_trending', 
            y='frequency', 
            labels={'days_on_trending': 'Days', 'frequency': 'Videos'},
            title='Phân bố video theo số ngày trên xu hướng',
            color='frequency',
            color_continuous_scale='Viridis'
        )
        time_on_trending_filtered = time_on_trending.drop(columns=['trending_time'])
        columns = [{"name": col, "id": col} for col in time_on_trending_filtered.columns]
        table_data = time_on_trending_filtered.to_dict('records')


    elif selected_chart == 'other1':
        fig = px.pie(category_pie,
                    values='count',
                    names=category_pie.index,
                    title='Tỉ lệ phần trăm của từng chủ đề')
        category_pie_reset = category_full.reset_index().drop(columns=['index'])
        columns = [{"name": col, "id": col} for col in category_pie_reset.columns]
        table_data = category_pie_reset.to_dict('records')


    elif selected_chart == 'other2':
        if selected_metric != 'like_to_view'  and selected_metric != 'comment_to_view':
            selected_metric = 'like_to_view'
        interaction_options = [
            {'label': 'Tỷ lệ Like/Views', 'value': 'like_to_view'},
            {'label': 'Tỷ lệ Comment/Views', 'value': 'comment_to_view'}
        ]
        interaction_style = {'margin-top': '10px', 'display': 'block'}
        
        if selected_metric == 'like_to_view':
            general_category_reset=general_category
            general_category_reset['like_to_view'] = general_category_reset['like_to_view'].round(2)
            general_category_reset = general_category_reset.sort_values(by='like_to_view')
            fig = px.bar(
                general_category_reset,
                x='category',
                y='like_to_view',
                labels={'category': 'Danh mục', 'like_to_view': 'Tỷ lệ Like/Views'},
                title='Tỷ lệ Like/Views theo Danh mục',
                text_auto=True,
                color='like_to_view',
                color_continuous_scale='Spectral'
            ).update_layout(xaxis_tickangle=45)
            fig.update_traces(
                textposition='outside'  
            )
            columns = [{"name": "Danh mục", "id": "category"},
                       {"name": "Tỷ lệ Like/Views (%)", "id": "like_to_view"}]
            table_data = general_category_reset.to_dict('records')

        elif selected_metric == 'comment_to_view':
            general_category['comment_to_view'] = general_category['comment_to_view'].round(3)
            general_category_reset_1 = general_category.sort_values(by='comment_to_view')
            fig = px.bar(
                general_category_reset_1,
                x='category',
                y='comment_to_view',
                labels={'category': 'Danh mục', 'comment_to_view': 'Tỷ lệ Comment/Views'},
                title='Tỷ lệ Comment/Views theo Danh mục',
                text_auto=True,
                color='comment_to_view',
                color_continuous_scale='Spectral'
            ).update_layout(xaxis_tickangle=45)
            fig.update_traces(
                textposition='outside'  
            )
            columns = [{"name": "Danh mục", "id": "category"},
                       {"name": "Tỷ lệ Comment/Views (%)", "id": "comment_to_view"}]
            table_data = general_category_reset_1.to_dict('records')


    elif selected_chart == 'other3':
        fig = px.bar(trending_category,
                    x=trending_category['trending_time'],
                    y=trending_category.index,
                    labels={'x': 'Category', 'y': 'Average Trending Time (hours)'},
                    title='Thời gian trung bình trên xu hướng theo danh mục')
        trending_category_reset = trending_category.reset_index()
        columns = [{"name": col, "id": col} for col in trending_category_reset.columns]
        table_data = trending_category_reset.to_dict('records')


    elif selected_chart == 'other4':
        fig = px.bar(tags_df_sort,
                   x='count',
                   y='tag',
                   title='Các từ khóa phổ biến',)
        tags_df_sort_reset = tags_df_sort.reset_index().drop(columns=['index'])
        columns = [{"name": col, "id": col} for col in tags_df_sort_reset.columns]
        table_data = tags_df_sort_reset.to_dict('records')


    # elif selected_chart == 'other5':
    #     fig = px.bar(len_df_to_plot,
    #                x='len',y='count',
    #                color='count',
    #                color_continuous_scale='Viridis')
    #     len_df_to_plot_reset = len_df_to_plot.reset_index().drop(columns=['index'])
    #     columns = [{"name": col, "id": col} for col in len_df_to_plot_reset.columns]
    #     table_data = len_df_to_plot_reset.to_dict('records')


    elif selected_chart == 'other6':
        fig = px.bar(duration_videos_long, 
                    x='count', 
                    y='time',
                    orientation='h', 
                    color='time', 
                    animation_frame='week', 
                    title='Thời lượng video theo thời gian',
                    labels={'count': 'Number of Videos', 'time': 'Duration Range', 'week': 'Week'},
                    text='count')

        fig.update_layout(
            xaxis_title="Number of Videos", 
            yaxis_title="Duration Range",
            showlegend=False
        )

        fig.update_layout(
            updatemenus=[{
                'buttons': [
                    {
                        'args': [None, {
                            'frame': {'duration': 100, 'redraw': True},  
                            'fromcurrent': True,
                            'transition': {'duration': 100, 'easing': 'cubic-in-out'} 
                        }], 'label': '&#9654;', 'method': 'animate'
                    },
                    {
                        'args': [[None], {
                            'frame': {'duration': 0, 'redraw': True},
                            'mode': 'immediate',
                            'transition': {'duration': 0}
                        }], 'label': 'Pause', 'method': 'animate'
                    }
                ]
            }]
        )
        duration_videos_long_reset = duration_videos_long.reset_index().drop(columns=['index'])
        columns = [{"name": col, "id": col} for col in duration_videos_long_reset.columns]
        table_data = duration_videos_long_reset.to_dict('records')
        interaction_style = {'display': 'none'}
        table_style = {'display': 'none'}
        

    elif selected_chart == 'other7':
        fig = px.bar(
        category_area_long,
        x='count', 
        y='categories',
        orientation='h', 
        color='categories', 
        animation_frame='month', 
        title='Chủ đề của video trending theo thời gian',
        labels={
            'count': 'Number of Videos', 
            'categories': 'Duration Range', 
            'month': 'Week'
        },
        text='count',
    )
        category_area_long['categories'] = pd.Categorical(category_area_long['categories'], 
        categories=category_area_long.groupby('categories')['count'].sum().sort_values(ascending=False).index)
        fig.update_layout(yaxis={'categoryorder':'total ascending'})
        fig.update_layout(showlegend=False)
        fig.update_traces(textposition='outside')
        columns = []  
        table_data = []  
        interaction_style = {'display': 'none'}
        table_style = {'display': 'none'}

    elif selected_chart == 'other8':
        general_month['snapshot_date'] = general_month['snapshot_date'].astype(str)
        general_month['view_count'] = general_month['view_count'].round(2)
        general_month['comment_count'] = general_month['comment_count'].round(2)
        general_month['like_count'] = general_month['like_count'].round(2)

        if selected_metric != 'view_count' and selected_metric != 'like_count'and selected_metric != 'comment_count':
            selected_metric = 'view_count'  
        interaction_options = [
            {'label': 'View Count', 'value': 'view_count'},
            {'label': 'Like Count', 'value': 'like_count'},
            {'label': 'Comment Count', 'value': 'comment_count'}
        ]
        interaction_style = {'margin-top': '10px', 'display': 'block'}
        
        if selected_metric == 'view_count':
            fig = px.bar(
                general_month,
                x='snapshot_date',
                y='view_count',
                title='Monthly View Count',
                labels={'snapshot_date': 'Month', 'view_count': 'Average View Count'},
                text_auto=True
            ).update_layout(xaxis_tickangle=45)
            fig.update_traces(
                textposition='outside'  
            )
            columns = [{"name": "Month", "id": "snapshot_date"},
                    {"name": "Average View Count", "id": "view_count"}]
            table_data = general_month[['snapshot_date', 'view_count']].to_dict('records')

        elif selected_metric == 'like_count':
            fig = px.bar(
                general_month,
                x='snapshot_date',
                y='like_count',
                title='Monthly Like Count',
                labels={'snapshot_date': 'Month', 'like_count': 'Average Like Count'},
                text_auto=True
            ).update_layout(xaxis_tickangle=45)
            fig.update_traces(
                textposition='outside'  
            )
            columns = [{"name": "Month", "id": "snapshot_date"},
                    {"name": "Average Like Count", "id": "like_count"}]
            table_data = general_month[['snapshot_date', 'like_count']].to_dict('records')

        elif selected_metric == 'comment_count':
            fig = px.bar(
                general_month,
                x='snapshot_date',
                y='comment_count',
                title='Monthly Comment Count',
                labels={'snapshot_date': 'Month', 'comment_count': 'Average Comment Count'},
                text_auto=True
            ).update_layout(xaxis_tickangle=45)
            fig.update_traces(
                textposition='outside'  
            )
            columns = [{"name": "Month", "id": "snapshot_date"},
                    {"name": "Average Comment Count", "id": "comment_count"}]
            table_data = general_month[['snapshot_date', 'comment_count']].to_dict('records')


    return fig, columns, table_data, interaction_style, interaction_options, table_style


if __name__ == '__main__':
    app.run_server(debug=True)
